In [2]:

import pandas as pd
import numpy as np


#upload this file /Users/tommi/Desktop/Uni_documents/Data/Dragon-Portfolio 2015.csv using the Dates as index
df = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/Dragon-Portfolio 2015.csv', index_col='Dates', parse_dates=True)





In [3]:
df.head()  


,SPXT Index,SPY US Equity,IVV US Equity,VOO US Equity,URTH US Equity,ACWI US Equity,LT11TRUU Index,IEF US Equity,BND US Equity,AGG US Equity,...,DBC US Equity,EHFI451 Index,ALTBX US Equity,VIXY US Equity,XVZ US Equity,VXX US Equity,VXZ US Equity,UVXY US Equity,VIXM US Equity,ZROZ US Equity.1
Dates,,,,,,,,,,,,,,,,,,,,,
02/01/2015,3768.68,205.43,206.73,188.40,71.650,58.36,634.31,106.53,82.65,110.43,...,18.23,NaN,10.38,32976.0,28.6001,NaN,NaN,30412500.0,63.37,121.49
05/01/2015,3700.03,201.72,203.10,185.09,69.970,57.21,644.88,107.18,82.89,110.67,...,17.97,NaN,10.40,35440.0,29.2499,NaN,NaN,34787500.0,66.60,124.72
06/01/2015,3667.14,199.82,201.27,183.27,69.040,56.62,654.82,107.90,83.13,110.95,...,17.80,NaN,10.44,36208.0,29.1000,NaN,NaN,36337500.0,66.90,127.98
07/01/2015,3710.94,202.31,203.76,185.56,69.740,57.36,656.15,107.88,83.18,110.93,...,17.69,NaN,10.44,35008.0,28.7900,NaN,NaN,34037500.0,65.25,128.21
08/01/2015,3777.40,205.90,207.40,188.82,71.288,58.26,646.65,107.44,83.05,110.76,...,17.76,NaN,10.43,32768.0,28.3499,NaN,NaN,29850000.0,62.56,125.09


In [4]:
df.columns

Index(['SPXT Index', 'SPY US Equity', 'IVV US Equity', 'VOO US Equity',
       'URTH US Equity', 'ACWI US Equity', 'LT11TRUU Index', 'IEF US Equity',
       'BND US Equity', 'AGG US Equity', 'TLH US Equity', 'ZROZ US Equity',
       'GOLDLNPM Index', 'GLD US Equity', 'IAU US Equity', 'PHYS US Equity',
       'XAU Curncy', 'HFRXMADC Index', 'HFND US Equity', 'KMLM US Equity',
       'CTA US Equity', 'DBMF US Equity', 'PDBC US Equity', 'DBC US Equity',
       'EHFI451 Index', 'ALTBX US Equity', 'VIXY US Equity', 'XVZ US Equity',
       'VXX US Equity', 'VXZ US Equity', 'UVXY US Equity', 'VIXM US Equity',
       'ZROZ US Equity.1'],
      dtype='object')

In [10]:
import pandas as pd
import statsmodels.api as sm



# Group assets by class
asset_classes = {
    "Equities": {
        "benchmark": "SPXT Index",
        "proxies": ["SPY US Equity", "IVV US Equity", "VOO US Equity", "URTH US Equity", "ACWI US Equity"]
    },
    "Fixed Income": {
        "benchmark": "LT11TRUU Index",
        "proxies": ["IEF US Equity", "BND US Equity", "AGG US Equity", "TLH US Equity", "ZROZ US Equity"]
    },
    "Gold": {
        "benchmark": "GOLDLNPM Index",
        "proxies": ["GLD US Equity", "IAU US Equity", "PHYS US Equity", "XAU Curncy"]
    },
    "Commodity Trend Following": {
        "benchmark": "HFRXMADC Index",
        "proxies": ["HFND US Equity", "KMLM US Equity", "CTA US Equity", "DBMF US Equity", "PDBC US Equity", "DBC US Equity"]
    },
    "Active Long Volatility": {
        "benchmark": "EHFI451 Index",
        "proxies": ["ALTBX US Equity", "VIXY US Equity", "XVZ US Equity", "VXX US Equity", "VXZ US Equity", "UVXY US Equity", "VIXM US Equity"]
    }
}

# Function to run regressions and find best proxy
def find_best_proxy(df, benchmark, proxies):
    results = {}
    
    for proxy in proxies:
        # Drop NaNs to avoid errors
        temp_df = df[[benchmark, proxy]].dropna()
        
        # Ensure we have enough data points
        if temp_df.shape[0] < 30:  # Require at least 30 observations
            continue
        
        # Define independent (X) and dependent (Y) variables
        X = sm.add_constant(temp_df[proxy])  # Add constant for intercept
        Y = temp_df[benchmark]
        
        # Run regression
        model = sm.OLS(Y, X).fit()
        results[proxy] = model.rsquared  # Store R² score

    # Find best proxy
    if results:
        best_proxy = max(results, key=results.get)
        return best_proxy, results[best_proxy]
    else:
        return None, None

# Run regression for each asset class and find best proxy
best_proxies = {}

for asset_class, data in asset_classes.items():
    best_proxy, r_squared = find_best_proxy(df, data["benchmark"], data["proxies"])
    best_proxies[asset_class] = (best_proxy, r_squared)

# Print results
print("\n📌 **Best Proxies for Each Asset Class:**\n")
for asset_class, (proxy, r_squared) in best_proxies.items():
    print(f"{asset_class}: Best Proxy = {proxy} (R² = {r_squared:.4f})" if proxy else f"{asset_class}: No valid proxy found")



📌 **Best Proxies for Each Asset Class:**

Equities: Best Proxy = SPY US Equity (R² = 0.9988)
Fixed Income: Best Proxy = ZROZ US Equity (R² = 0.8816)
Gold: Best Proxy = XAU Curncy (R² = 0.9996)
Commodity Trend Following: No valid proxy found
Active Long Volatility: No valid proxy found
